In [1]:
#@title Pip install modules
%pip install -Uqqq pip --progress-bar off
%pip install -qqq bitsandbytes==0.39.0 --progress-bar off
%pip install -qqq torch==2.0.1 --progress-bar off
%pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
%pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
%pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
%pip install -qqq -U datasets==2.12.0 --progress-bar off
%pip install -qqq -U loralib==0.1.1 --progress-bar off
%pip install -qqq -U einops==0.6.1 --progress-bar off
%pip install -qqq -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

notebook_login()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  

In [3]:
#@title Dataset
dataset = [
    {
        "input": "You wake up in a mysterious forest.",
        "output": "Look around to assess your surroundings."
    },
    {
        "input": "A sudden gust of wind blows open the door.",
        "output": "Investigate the source of the wind."
    },
    {
        "input": "You enter a dimly lit cave.",
        "output": "Proceed cautiously deeper into the cave."
    },
    {
        "input": "You stumble upon a rickety old bridge.",
        "output": "Test the bridge's stability before crossing."
    },
    {
        "input": "A distant howl sends chills down your spine.",
        "output": "Determine the direction of the howl and decide whether to investigate."
    },
    {
        "input": "You find a torn map on the ground.",
        "output": "Examine the map and try to piece it back together."
    },
    {
        "input": "The sound of running water catches your attention.",
        "output": "Follow the sound to discover its source."
    },
    {
        "input": "You notice a strange symbol etched into a tree trunk.",
        "output": "Inspect the symbol and try to decipher its meaning."
    },
    {
        "input": "You stumble upon an abandoned cabin.",
        "output": "Enter the cabin to see what's inside."
    },
    {
        "input": "You spot a glimmering object in the distance.",
        "output": "Approach the object to get a closer look."
    },
    {
        "input": "You hear faint whispers echoing through the corridor.",
        "output": "Listen carefully to the whispers and try to make out what they're saying."
    },
    {
        "input": "You encounter a deep, dark well.",
        "output": "Peer into the well to see if there's anything at the bottom."
    },
    {
        "input": "A mysterious figure appears in front of you.",
        "output": "Initiate a conversation with the figure to learn more."
    },
    {
        "input": "You find yourself in a room filled with mirrors.",
        "output": "Inspect the mirrors and see if there's anything unusual about them."
    },
    {
        "input": "You stumble upon a hidden trapdoor in the floor.",
        "output": "Open the trapdoor and see where it leads."
    },
    {
        "input": "A riddle is written on the wall before you.",
        "output": "Attempt to solve the riddle and uncover its meaning."
    },
    {
        "input": "You come across a locked gate blocking your path.",
        "output": "Search for a way to unlock the gate and continue forward."
    },
    {
        "input": "A mysterious journal lies on a dusty table.",
        "output": "Read the journal to see if it contains any valuable information."
    },
    {
        "input": "A thick fog obscures your vision.",
        "output": "Try to disperse the fog or find a way to navigate through it."
    },
    {
        "input": "You hear the sound of footsteps behind you.",
        "output": "Turn around to see who or what is approaching."
    },
    {
        "input": "A gentle breeze carries an enticing aroma.",
        "output": "Follow the scent to discover its source."
    },
    {
        "input": "You notice a glint of light coming from a high window.",
        "output": "Find a way to reach the window and investigate."
    },
    {
        "input": "A mysterious voice whispers a cryptic message in your ear.",
        "output": "Listen carefully and try to decipher the message's meaning."
    },
    {
        "input": "You stumble upon a forgotten graveyard.",
        "output": "Explore the graveyard and search for any clues or items of interest."
    },
    {
        "input": "A sudden earthquake shakes the ground beneath you.",
        "output": "Find a stable spot or take cover until the shaking subsides."
    },
    {
        "input": "You find a torn page from a diary.",
        "output": "Read the torn page and see if it provides any useful information."
    },
    {
        "input": "A beam of moonlight illuminates a hidden path.",
        "output": "Follow the moonlit path and see where it leads."
    },
    {
        "input": "You discover a locked door with strange symbols engraved on it.",
        "output": "Examine the symbols and try to decipher a way to unlock the door."
    },
    {
        "input": "A stray cat crosses your path and stares at you.",
        "output": "Interact with the cat and see if it leads you somewhere."
    },
    {
        "input": "You find a worn-out photograph lying on the ground.",
        "output": "Pick up the photograph and examine its contents."
    },
    {
        "input": "A sudden downpour drenches the area around you.",
        "output": "Find shelter or a way to protect yourself from the rain."
    },
    {
        "input": "You notice a faint glimmer underwater in a nearby lake.",
        "output": "Dive into the lake and investigate the glimmer."
    },
    {
        "input": "A trail of footprints leads into a dense forest.",
        "output": "Follow the footprints and see where they lead."
    },
    {
        "input": "You come across a broken-down carriage on the side of the road.",
        "output": "Search the carriage for any useful items or information."
    },
    {
        "input": "A swarm of fireflies lights up the night sky.",
        "output": "Observe the fireflies and see if they form a pattern or lead you somewhere."
    },
    {
        "input": "You find a mysterious potion vial hidden behind a loose brick.",
        "output": "Inspect the potion vial and consider its potential effects."
    },
    {
        "input": "A mysterious amulet glows softly in a display case.",
        "output": "Attempt to retrieve the amulet from the display case."
    },
    {
        "input": "You encounter a wise old hermit living in seclusion.",
        "output": "Engage in conversation with the hermit and seek their guidance."
    },
    {
        "input": "A gust of wind blows out the candle, plunging the room into darkness.",
        "output": "Light the candle again or find another source of illumination."
    },
    {
        "input": "You notice a trail of breadcrumbs leading into a thick forest.",
        "output": "Follow the trail of breadcrumbs and see where it takes you."
    },
    {
        "input": "A peculiar statue stands in the center of a courtyard.",
        "output": "Examine the statue closely and look for any hidden mechanisms or inscriptions."
    },
    {
        "input": "You come across a broken bridge spanning a deep ravine.",
        "output": "Search for a way to repair the bridge or find an alternate route."
    },
    {
        "input": "A faint melody fills the air, drawing your attention.",
        "output": "Follow the melody and try to find its source."
    },
    {
        "input": "You find a tattered treasure map hidden inside an old book.",
        "output": "Study the treasure map and plan your journey to find the hidden treasure."
    },
    {
        "input": "A friendly squirrel approaches you, holding a small object.",
        "output": "Interact with the squirrel and see what it wants to show you."
    },
    {
        "input": "You stumble upon a forgotten underground chamber.",
        "output": "Explore the chamber and search for any clues or ancient artifacts."
    },
    {
        "input": "A distant roar echoes through the mountains.",
        "output": "Ascend the nearest peak to get a better view and identify the source of the roar."
    },
    {
        "input": "You find a diary filled with cryptic writings.",
        "output": "Decipher the cryptic writings in the diary and uncover its secrets."
    },
    {
        "input": "A dense mist envelops the area, making it difficult to see.",
        "output": "Try to disperse the mist or find a way to navigate through it safely."
    },
    {
        "input": "You encounter a talking statue that offers a riddle.",
        "output": "Attempt to solve the statue's riddle to unlock its hidden knowledge."
    },
    {
        "input": "A shimmering portal appears before you.",
        "output": "Step through the portal to explore a new realm or dimension."
    },
    {
        "input": "You find a broken compass on the ground.",
        "output": "Try to repair the compass or find an alternative method of navigation."
    },
    {
        "input": "A flock of birds suddenly takes flight, heading in a specific direction.",
        "output": "Follow the path of the birds and see where they lead."
    },
    {
        "input": "You come across a magical fountain with glowing water.",
        "output": "Drink from the fountain or collect some of the glowing water for later use."
    },
    {
        "input": "A group of friendly fairies offer their assistance.",
        "output": "Accept the fairies' help and let them guide you through the enchanted realm."
    },
    {
        "input": "You notice a peculiar symbol carved into a tree trunk.",
        "output": "Investigate the symbol and see if it holds any significance or leads to a hidden path."
    },
    {
        "input": "A mysterious package arrives addressed to you.",
        "output": "Open the package and uncover its contents, being cautious of any potential traps."
    },
    {
        "input": "You stumble upon a magical book with a pulsing glow.",
        "output": "Open the book and explore its magical pages, being prepared for any consequences."
    },
    {
        "input": "A chilling breeze whispers secrets in your ear.",
        "output": "Listen closely and try to decipher the whispered secrets."
    },
    {
        "input": "You find a broken sword lying next to a fallen warrior.",
        "output": "Inspect the sword and see if it can be repaired or salvaged."
    },
    {
        "input": "A mysterious symbol is etched into the stone floor.",
        "output": "Trace your fingers along the symbol and see if it triggers any response."
    },
    {
        "input": "You come across a magical artifact hidden inside an ancient chest.",
        "output": "Carefully examine the artifact and try to harness its powers."
    },
    {
        "input": "A sudden silence fills the air, as if time has frozen.",
        "output": "Investigate the cause of the silence and observe your surroundings."
    },
    {
        "input": "You notice a shimmering portal to another realm.",
        "output": "Summon the courage to step through the portal and embark on a new adventure."
    },
    {
        "input": "A mischievous spirit taunts you from the shadows.",
        "output": "Engage with the spirit and see if you can gain its trust or appease it."
    },
    {
        "input": "You find a hidden compartment behind a painting on the wall.",
        "output": "Open the compartment and discover what secrets it holds."
    },
    {
        "input": "A celestial alignment reveals a hidden path in the night sky.",
        "output": "Follow the pattern of stars and traverse the celestial path."
    },
    {
        "input": "You stumble upon a forgotten ancient temple.",
        "output": "Explore the temple and unlock its mysteries."
    },
    {
        "input": "A mysterious fog engulfs the area, distorting reality.",
        "output": "Navigate through the fog using your instincts and senses."
    },
    {
        "input": "You find a cryptic message carved into the bark of a tree.",
        "output": "Decipher the message and see if it leads to a hidden treasure or revelation."
    },
    {
        "input": "A spectral figure beckons you from afar.",
        "output": "Approach the figure cautiously and find out what it desires."
    },
    {
        "input": "You discover a hidden passage behind a bookshelf.",
        "output": "Push aside the bookshelf and venture into the hidden passage."
    },
    {
        "input": "A wise oracle offers you a glimpse into your future.",
        "output": "Listen to the oracle's prophecy and contemplate its meaning."
    },
    {
        "input": "You stumble upon a magical portal that allows time travel.",
        "output": "Step through the portal and explore different time periods."
    },
    {
        "input": "A haunting melody fills the air, evoking intense emotions.",
        "output": "Follow the melody and uncover the source of its haunting beauty."
    },
    {
        "input": "You find a peculiar plant with glowing, luminescent petals.",
        "output": "Interact with the plant and observe its unique properties."
    },
    {
        "input": "A mysterious inscription appears on your hand.",
        "output": "Examine the inscription and try to interpret its significance."
    },
    {
        "input": "You encounter a group of friendly forest creatures.",
        "output": "Interact with the creatures and see if they offer any assistance or guidance."
    },
    {
        "input": "A thunderstorm brews on the horizon, casting an eerie glow.",
        "output": "Seek shelter or embrace the storm's energy and harness its power."
    },
    {
        "input": "You find an ancient scroll hidden within a dusty library.",
        "output": "Carefully unravel the scroll and uncover its ancient knowledge."
    },
    {
        "input": "A series of cryptic symbols line the walls of a secret chamber.",
        "output": "Decipher the symbols and unlock the chamber's secrets."
    },
    {
        "input": "You stumble upon a forgotten village overrun by nature.",
        "output": "Investigate the village and uncover its history and inhabitants."
    },
    {
        "input": "A celestial phenomenon illuminates the night sky with vibrant colors.",
        "output": "Marvel at the celestial display and contemplate its significance."
    },
    {
        "input": "You encounter a mythical creature believed to grant wishes.",
        "output": "Engage with the creature and make a wish, being mindful of the consequences."
    },
    {
        "input": "A mysterious figure in a cloak guides you towards an unknown destination.",
        "output": "Follow the figure and trust their guidance to lead you to your destiny."
    },
    {
        "input": "You discover a hidden chamber filled with ancient relics.",
        "output": "Examine the relics and try to unravel the stories they hold."
    },
    {
        "input": "A labyrinthine maze stretches out before you.",
        "output": "Navigate the maze, making strategic choices to find your way out."
    },
    {
        "input": "You find a magical talisman pulsating with energy.",
        "output": "Grasp the talisman and feel its power course through your veins."
    },
    {
        "input": "A celestial body streaks across the night sky, leaving a trail of shimmering stardust.",
        "output": "Follow the trail of stardust and see where it leads."
    },
    {
        "input": "You come across a forgotten kingdom lost in time.",
        "output": "Explore the kingdom and uncover its rich history and secrets."
    },
    {
        "input": "A mystical door materializes in front of you, beckoning you to step through.",
        "output": "Summon your courage and step through the mystical door to embark on a new adventure."
    },
    {
        "input": "You stumble upon a hidden oasis in the middle of a barren desert.",
        "output": "Rest and replenish your supplies in the oasis before continuing your journey."
    },
    {
        "input": "A shimmering veil separates the mortal realm from the realm of magic.",
        "output": "Find a way to pierce through the veil and explore the realm of magic."
    },
    {
        "input": "You discover an enchanted forest where time flows differently.",
        "output": "Enter the enchanted forest and experience its magical time-warping effects."
    },
    {
        "input": "A mystical fog descends, revealing glimpses of a parallel realm.",
        "output": "Venture into the parallel realm and uncover its connection to your own world."
    },
    {
        "input": "You find an ancient artifact said to possess incredible powers.",
        "output": "Handle the artifact with caution and discover its true potential."
    },
    {
        "input": "A celestial alignment opens a portal to a realm of celestial beings.",
        "output": "Step through the portal and interact with the celestial beings to gain their wisdom."
    },
    {
        "input": "You stumble upon a hidden underground city filled with advanced technology.",
        "output": "Explore the underground city and unravel its futuristic mysteries."
    },
    {
        "input": "A mysterious mist envelopes the forest, distorting sounds and shapes.",
        "output": "Navigate through the mist, relying on your instincts and senses."
    },
    {
        "input": "You discover a hidden society of shape-shifting creatures.",
        "output": "Interact with the shape-shifters and learn their secrets."
    },
    {
        "input": "A legendary artifact is rumored to be hidden within a treacherous mountain range.",
        "output": "Embark on a perilous journey to locate and claim the legendary artifact."
    },
    {
        "input": "You encounter a sentient tree that imparts ancient wisdom.",
        "output": "Engage in conversation with the sentient tree and absorb its profound knowledge."
    },
    {
        "input": "An ethereal song echoes through a magical valley, captivating your attention.",
        "output": "Follow the melodic song and uncover the source of its enchantment."
    },
    {
        "input": "You stumble upon a hidden academy where magic is taught and mastered.",
        "output": "Enroll in the academy and embark on a journey to harness your magical abilities."
    },
    {
        "input": "A mysterious misty portal appears, offering passage to an unknown realm.",
        "output": "Brave the portal and explore the unfamiliar realm, eager for new discoveries."
    },
    {
        "input": "You find an ancient tome filled with forbidden spells and incantations.",
        "output": "Delve into the forbidden knowledge within the tome, treading carefully on the path of magic."
    },
    {
        "input": "A time-traveling device comes into your possession, granting the ability to visit different eras.",
        "output": "Utilize the time-traveling device and explore pivotal moments in history."
    },
    {
        "input": "You encounter a mischievous sprite who leads you on a whimsical adventure.",
        "output": "Follow the sprite and immerse yourself in a world of wonder and surprises."
    },
    {
        "input": "A legendary sword is said to be embedded within a mythical stone.",
        "output": "Attempt to extract the sword from the stone, proving your worthiness and fulfilling your destiny."
    },
    {
        "input": "You discover a hidden sanctuary where mythical creatures seek refuge.",
        "output": "Gain the trust of the mythical creatures and become part of their extraordinary world."
    },
    {
        "input": "A powerful enchantress offers you a chance to reshape your destiny.",
        "output": "Accept the enchantress's offer and navigate the consequences of altering your fate."
    },
    {
        "input": "You stumble upon a lost civilization thriving deep beneath the ocean's surface.",
        "output": "Explore the underwater civilization and uncover the mysteries of their advanced society."
    },
    {
        "input": "A riddle-guarded treasure awaits at the heart of an ancient labyrinth.",
        "output": "Solve the riddles and navigate the labyrinth to claim the hidden treasure."
    },
    {
        "input": "You find a mystical artifact that grants temporary elemental powers.",
        "output": "Harness the powers of the mystical artifact and wield the elements at your command."
    },
    {
        "input": "A mysterious traveler presents you with a map leading to a forgotten realm.",
        "output": "Follow the map's guidance and venture into the forgotten realm to unveil its secrets."
    },
    {
        "input": "You encounter a magical creature capable of granting a single wish.",
        "output": "Carefully consider your wish and interact with the creature to make it come true."
    },
    {
        "input": "A legendary phoenix is rumored to reside atop a treacherous mountain peak.",
        "output": "Embark on a challenging ascent to encounter the majestic phoenix and unlock its wisdom."
    },
    {
        "input": "You discover an enchanted garden filled with rare and mythical plants.",
        "output": "Explore the enchanted garden and learn about the magical properties of its flora."
    },
    {
        "input": "A mysterious mask reveals hidden truths to those who wear it.",
        "output": "Put on the mask and uncover the hidden truths that lie beyond ordinary perception."
    },
    {
        "input": "You stumble upon a time-worn map leading to a buried pirate treasure.",
        "output": "Embark on a pirate adventure, following the map to unearth the hidden treasure."
    },
    {
        "input": "A celestial event grants you temporary powers of telekinesis.",
        "output": "Experiment with your newfound telekinetic abilities and explore their limits."
    },
    {
        "input": "You discover a sentient magical creature in need of your assistance.",
        "output": "Offer your help to the magical creature and embark on a quest to fulfill its needs."
    },
    {
        "input": "A mysterious phenomenon grants you the ability to understand all languages.",
        "output": "Embrace your newfound linguistic prowess and engage in conversations with diverse beings."
    },
    {
        "input": "You encounter a whimsical traveling circus with extraordinary performers.",
        "output": "Immerse yourself in the wonders of the circus and witness awe-inspiring performances."
    },
    {
        "input": "A forgotten prophecy speaks of a chosen hero destined to save the realm.",
        "output": "Embrace your role as the chosen hero and embark on a quest to fulfill the prophecy."
    },
    {
        "input": "You discover a hidden portal that leads to an alternate dimension.",
        "output": "Step through the portal and explore the wonders and dangers of the alternate dimension."
    },
    {
        "input": "A mystical crystal possesses the power to grant a single wish.",
        "output": "Harness the power of the crystal and carefully choose your one heartfelt wish."
    },
    {
        "input": "You stumble upon an ancient language written on weathered stone tablets.",
        "output": "Decipher the ancient language and unlock the ancient knowledge contained within."
    },
    {
        "input": "A spirit guide appears to offer guidance and wisdom on your journey.",
        "output": "Engage in conversation with the spirit guide and seek its profound insights."
    },
    {
        "input": "You find an enchanted musical instrument with the power to manipulate emotions.",
        "output": "Play the enchanted instrument and explore the realm of emotions it unlocks."
    },
    {
        "input": "A mystical doorway materializes, leading to a realm of dreams and imagination.",
        "output": "Step through the mystical doorway and experience the limitless realm of dreams."
    },
    {
        "input": "You encounter a group of legendary warriors training in a secluded mountain temple.",
        "output": "Join the legendary warriors in their training and learn their ancient fighting techniques."
    },
    {
        "input": "A magical amulet allows you to communicate with animals and understand their language.",
        "output": "Utilize the power of the amulet to converse with animals and gain their insights."
    },
    {
        "input": "You stumble upon a hidden library filled with books that chronicle forgotten history.",
        "output": "Explore the hidden library and delve into the forgotten stories of the past."
    },
    {
        "input": "A powerful storm elemental beckons you to harness its energy and become its champion.",
        "output": "Accept the elemental's challenge and learn to wield the power of storms."
    },
    {
        "input": "You discover a secret society dedicated to preserving ancient mystical knowledge.",
        "output": "Become initiated into the secret society and learn the sacred teachings of mysticism."
    },
    {
        "input": "A celestial alignment grants you temporary control over celestial bodies.",
        "output": "Harness the power of the celestial alignment and manipulate the movements of stars and planets."
    },
    {
        "input": "You stumble upon an enchanted mirror that reveals glimpses of other dimensions.",
        "output": "Peer into the enchanted mirror and witness the wonders and mysteries of other dimensions."
    },
    {
        "input": "A mystical phenomenon bestows the ability to communicate with inanimate objects.",
        "output": "Embrace the gift of communication and engage in conversations with objects around you."
    },
    {
        "input": "You discover a hidden underground network of tunnels inhabited by a secretive society.",
        "output": "Explore the underground tunnels and uncover the secrets and purpose of the society."
    },
    {
        "input": "A magical compass leads you on a quest to find a legendary lost city.",
        "output": "Follow the guidance of the magical compass and embark on a journey to rediscover the lost city."
    },
    {
        "input": "You encounter a group of ancient guardians tasked with protecting a sacred artifact.",
        "output": "Gain the trust of the guardians and assist them in safeguarding the sacred artifact."
    },
    {
        "input": "A mysterious fog transports you to a realm of dreams and illusions.",
        "output": "Navigate through the realm of dreams, distinguishing reality from illusion."
    },
    {
        "input": "You find a forgotten diary filled with the personal experiences of a long-lost adventurer.",
        "output": "Immerse yourself in the adventurer's stories and retrace their footsteps."
    },
    {
        "input": "A hidden underground waterfall conceals a gateway to a realm of magical creatures.",
        "output": "Enter the gateway behind the waterfall and encounter the enchanting realm of magical beings."
    },
    {
        "input": "You stumble upon a hidden alchemist's laboratory filled with peculiar potions and experiments.",
        "output": "Explore the alchemist's laboratory and unravel the mysteries of their concoctions and research."
    },
    {
        "input": "An ancient curse afflicts a nearby village, and its residents seek your help.",
        "output": "Aid the villagers in breaking the ancient curse and restoring peace and prosperity."
    },
    {
        "input": "You encounter a sentient, talking statue that guards a sacred knowledge.",
        "output": "Engage in conversation with the talking statue and unlock the wisdom it possesses."
    },
    {
        "input": "A mystical phenomenon grants you the power of invisibility for a limited time.",
        "output": "Harness the power of invisibility and explore the world unseen, observing and learning."
    },
    {
        "input": "You find a magical book that allows you to rewrite reality with every page turned.",
        "output": "Exercise caution and use the magical book's power wisely to shape a new reality."
    },
    {
        "input": "A legendary creature challenges you to a test of strength and skill.",
        "output": "Accept the creature's challenge and prove your worth through the test of strength and skill."
    },
    {
        "input": "You discover an ancient relic that opens portals to different dimensions.",
        "output": "Activate the ancient relic and explore the diverse dimensions it unveils."
    },
    {
        "input": "A spectral guide appears, offering to assist you on a perilous journey.",
        "output": "Accept the spectral guide's assistance and rely on their guidance during the challenging journey."
    },
    {
        "input": "You stumble upon a hidden society of elemental magic users.",
        "output": "Immerse yourself in the society of elemental magic users and learn their ancient arts."
    },
    {
        "input": "A mystical spring possesses the power to heal both physical and emotional wounds.",
        "output": "Approach the mystical spring and allow its healing waters to restore your well-being."
    },
    {
        "input": "You encounter a time-traveling merchant who offers rare and exotic artifacts.",
        "output": "Engage with the time-traveling merchant and acquire extraordinary artifacts from different eras."
    },
    {
        "input": "A celestial event grants you the ability to speak to the stars and receive cosmic wisdom.",
        "output": "Commune with the stars and seek their guidance to gain profound cosmic wisdom."
    },
    {
        "input": "You discover an enchanted mirror that reveals your true inner self.",
        "output": "Gaze into the enchanted mirror and confront your deepest fears and desires."
    },
    {
        "input": "A magical forest grows in the moonlight, housing mystical creatures and sentient flora.",
        "output": "Explore the enchanted forest, encountering extraordinary beings and conversing with the wise plants."
    },
    {
        "input": "You inherit an ancient family heirloom with unknown powers and a cryptic message.",
        "output": "Unlock the powers of the family heirloom and decipher the hidden message to unveil its purpose."
    },
    {
        "input": "A legendary phoenix chooses you as its destined companion and grants you its fiery abilities.",
        "output": "Embrace your role as the phoenix's companion and learn to wield the flames of its power."
    },
    {
        "input": "You stumble upon a hidden realm where time flows backward, unraveling past events.",
        "output": "Enter the realm of reversed time and witness the untold stories of the past, altering their outcomes."
    },
    {
        "input": "A mischievous fairy bestows you with a charm that grants wishes but at a cost.",
        "output": "Interact with the mischievous fairy and make wishes, mindful of the unexpected consequences."
    },
    {
        "input": "You discover a mythical portal guarded by ancient spirits, granting access to a realm of dreams.",
        "output": "Convince the ancient spirits of your worth and step through the portal into the realm of dreams."
    },
    {
        "input": "A magical phenomenon grants you the ability to communicate with mythical creatures.",
        "output": "Embrace the gift of communication and engage in conversations with majestic dragons, unicorns, and more."
    },
    {
        "input": "You stumble upon a hidden celestial observatory that reveals cosmic secrets.",
        "output": "Observe the celestial bodies and unravel the secrets of the universe within the hidden observatory."
    },
    {
        "input": "A dormant volcano awakens, unveiling a hidden labyrinth of fiery trials.",
        "output": "Venture into the volcanic labyrinth and overcome the fiery trials to prove your resilience."
    },
    {
        "input": "You encounter a mysterious oracle who holds the answers to your deepest questions.",
        "output": "Seek the oracle's guidance and ask the profound questions that weigh on your heart and mind."
    },
    {
        "input": "A magical storm engulfs the land, granting temporary elemental powers to those caught within.",
        "output": "Harness the power of the magical storm and wield the elements to protect and shape the world."
    },
    {
        "input": "You discover a forgotten language capable of casting spells and manipulating reality.",
        "output": "Learn the forgotten language of spells and experiment with its power to reshape your surroundings."
    },
    {
        "input": "An enchanted compass leads you on a quest to recover lost fragments of a shattered artifact.",
        "output": "Follow the compass's guidance and gather the scattered fragments to restore the shattered artifact's power."
    },
    {
        "input": "You stumble upon a hidden underground society where dreams manifest into reality.",
        "output": "Explore the underground society and witness the extraordinary creations born from the realm of dreams."
    },
    {
        "input": "A mystical mist surrounds a sacred grove, granting visions of the past, present, and future.",
        "output": "Enter the mist-shrouded grove and receive profound visions that offer glimpses into different timelines."
    },
    {
        "input": "You encounter a magical being capable of granting temporary shape-shifting abilities.",
        "output": "Interact with the magical being and experience the thrill of transforming into different creatures."
    },
    {
        "input": "A legendary library appears only to those with a thirst for knowledge and a pure heart.",
        "output": "Enter the legendary library and immerse yourself in its vast collection of wisdom and hidden truths."
    },
    {
        "input": "You find a sentient talking sword that seeks a worthy wielder for epic quests.",
        "output": "Claim the sentient sword and embark on heroic adventures as you fulfill its desire for glory."
    },
    {
        "input": "A mystical fog transports you to a realm of floating islands and ancient ruins.",
        "output": "Navigate the floating islands and uncover the ancient secrets hidden within the crumbling ruins."
    },
    {
        "input": "You stumble upon a hidden society of master illusionists, skilled in the art of deception.",
        "output": "Immerse yourself in the society of illusionists and learn the secrets of their mesmerizing illusions."
    },
    {
        "input": "A legendary forest guardian tests your worthiness to protect the realm's balance.",
        "output": "Face the challenges set by the forest guardian and prove yourself as a worthy protector of the realm."
    },
    {
        "input": "You discover a magical painting that transports you into the vibrant world it depicts.",
        "output": "Step into the magical painting and explore the fantastical world brought to life by its enchantment."
    },
    {
        "input": "A hidden realm of mythical creatures reveals itself during a rare celestial alignment.",
        "output": "Embark on a journey to the hidden realm and interact with the extraordinary creatures awakened by the celestial alignment."
    },
    {
        "input": "You encounter a time-traveling sorcerer who seeks your help in correcting a historical anomaly.",
        "output": "Join forces with the time-traveling sorcerer and venture through time to rectify the historical anomaly."
    },
    {
        "input": "A legendary treasure map is rumored to hold the key to immortality and unimaginable riches.",
        "output": "Embark on a treacherous quest to follow the treasure map and uncover the secrets of immortality and wealth."
    },
    {
        "input": "You discover an enchanted forest where trees whisper ancient prophecies and secrets.",
        "output": "Listen to the whispers of the enchanted forest and unravel the profound prophecies and hidden knowledge it holds."
    },
    {
        "input": "A mythical creature bestows you with the power to control the weather and influence natural elements.",
        "output": "Harness the power of weather manipulation and become a force to be reckoned with, shaping the elements to your will."
    },
    {
        "input": "You stumble upon a hidden realm of eternal winter ruled by a benevolent ice queen.",
        "output": "Navigate the frozen realm and develop a bond with the ice queen as you uncover the mysteries of the eternal winter."
    },
    {
        "input": "A magical mask allows you to see the true nature and intentions of those around you.",
        "output": "Wear the magical mask and peer into the hearts of others, discerning their true intentions and hidden motives."
    },
    {
        "input": "You encounter a mythical underwater city inhabited by merfolk and aquatic beings.",
        "output": "Dive into the depths of the underwater city and engage with the merfolk and other mystical creatures dwelling there."
    },
    {
        "input": "A forgotten prophecy foretells of a cataclysmic event that only you can prevent.",
        "output": "Embrace your destiny as the prophesied savior and embark on a race against time to avert the impending catastrophe."
    },
    {
        "input": "You discover an ancient artifact that grants the power to communicate with elemental spirits.",
        "output": "Activate the ancient artifact and forge connections with elemental spirits, gaining their guidance and elemental powers."
    },
    {
        "input": "A hidden society of shape-shifting wizards invites you to join their ranks and master their craft.",
        "output": "Accept the invitation to the society of shape-shifting wizards and delve into the art of transformation and illusion."
    },
    {
        "input": "You stumble upon a mythical creature sanctuary where endangered species find refuge.",
        "output": "Protect and care for the endangered mythical creatures within the sanctuary, fostering their preservation and well-being."
    },
    {
        "input": "A legendary cloak imbued with ancient enchantments grants you the ability to phase through solid objects.",
        "output": "Don the legendary cloak and explore the world by phasing through obstacles, uncovering hidden secrets along the way."
    },
    {
        "input": "You encounter a group of celestial beings seeking mortal champions to aid in their cosmic battles.",
        "output": "Become a chosen champion of the celestial beings and engage in epic cosmic battles against formidable adversaries."
    },
    {
        "input": "A magical flower blooms only once a century, possessing the power to grant one extraordinary wish.",
        "output": "Seek out the magical flower and make a wish that will shape the course of your destiny."
    },
    {
        "input": "You discover an ancient labyrinth guarded by riddles and puzzles, leading to a fabled treasure.",
        "output": "Solve the intricate riddles and navigate the perplexing labyrinth to claim the fabled treasure that awaits at its core."
    },
    {
        "input": "A mysterious celestial event bestows the gift of telepathy, allowing you to communicate with minds across distances.",
        "output": "Embrace the gift of telepathy and explore the vast possibilities of connecting with others through the power of the mind."
    },
    {
        "input": "You enter a bustling tavern filled with adventurers, each with their own tale to tell.",
        "output": "Join the adventurers in the tavern, share stories, and embark on epic quests together."
    },
    {
        "input": "You encounter a fearsome dragon guarding a hoard of treasures in its lair.",
        "output": "Face the mighty dragon in a battle of wits and strength, seeking to claim its treasure or forge an unlikely alliance."
    },
    {
        "input": "A mysterious old wizard tasks you with retrieving a powerful artifact from a perilous dungeon.",
        "output": "Accept the wizard's quest and venture into the treacherous depths of the dungeon, facing traps, monsters, and puzzles."
    },
    {
        "input": "You stumble upon an ancient prophecy foretelling the rise of a chosen hero to defeat an ancient evil.",
        "output": "Embrace your role as the chosen hero and gather allies to confront and vanquish the ancient evil threatening the realm."
    },
    {
        "input": "A guild of skilled thieves offers you the opportunity to join their ranks and become a master of stealth.",
        "output": "Prove your skills to the guild of thieves and embark on daring heists and covert missions, honing your abilities as a master of stealth."
    },
    {
        "input": "You discover a magical tome containing forbidden spells that hold great power but come at a dangerous price.",
        "output": "Delve into the forbidden spells of the magical tome, grappling with the ethical dilemma of wielding immense power with dire consequences."
    },
    {
        "input": "A curse transforms you into a monstrous creature, and you must find a way to break the curse before it consumes you.",
        "output": "Embark on a quest to lift the curse that afflicts you, seeking magical remedies, ancient rituals, or the aid of powerful beings."
    },
    {
        "input": "You are invited to a grand tournament where warriors from all realms gather to prove their valor.",
        "output": "Train, compete, and showcase your combat prowess in the grand tournament, earning glory, fame, and the respect of fellow warriors."
    },
    {
        "input": "A powerful artifact is shattered into fragments scattered across the land, and you are chosen to retrieve them and restore its power.",
        "output": "Embark on a quest to recover the shattered fragments of the artifact, facing challenges, adversaries, and unraveling the artifact's true purpose."
    },
    {
        "input": "You encounter a mysterious fortune teller who offers to reveal glimpses of your future and guide your path.",
        "output": "Consult the enigmatic fortune teller and heed their guidance as you navigate the twists and turns of fate in your adventurous journey."
    },
    {
        "input": "A divine entity grants you divine powers, charging you with the task of upholding justice and vanquishing evil.",
        "output": "Embrace your divine powers and become a champion of justice, venturing forth to confront evil in all its forms."
    },
    {
        "input": "You discover a hidden underground city inhabited by a race of ancient, wise beings.",
        "output": "Explore the underground city and seek the wisdom of the ancient beings, learning about their history, culture, and arcane knowledge."
    },
    {
        "input": "A legendary artifact, rumored to grant immense power, is said to be sealed within an impenetrable fortress.",
        "output": "Assemble a skilled team, devise a plan, and infiltrate the formidable fortress to claim the legendary artifact and harness its incredible power."
    },
    {
        "input": "You find a mysterious portal that transports you to a realm populated by fantastical creatures and mythical beings.",
        "output": "Venture into the realm beyond the portal, encountering strange creatures, engaging in diplomacy or conflict, and uncovering the secrets of the realm."
    },
    {
        "input": "A powerful curse threatens to consume the kingdom, and you must gather a group of allies to break the curse and restore balance.",
        "output": "Recruit a diverse group of allies, each with unique abilities, and embark on a perilous journey to break the curse and save the kingdom from impending doom."
    },
    {
        "input": "You are chosen as the champion of a deity and tasked with defending their sacred temple from dark forces.",
        "output": "Take up the mantle of the deity's champion, defending their sacred temple, unraveling the mysteries within, and vanquishing the encroaching darkness."
    },
    {
        "input": "A legendary artifact known as the 'Sword of Destiny' reveals itself, calling for a worthy hero to wield its power.",
        "output": "Answer the call of the 'Sword of Destiny' and embark on a quest to prove your worthiness, unlocking the artifact's true potential."
    },
    {
        "input": "You stumble upon a hidden library filled with ancient tomes and forbidden knowledge.",
        "output": "Immerse yourself in the hidden library's vast knowledge, delving into ancient tomes, deciphering cryptic texts, and uncovering the secrets that lie within."
    },
    {
        "input": "A powerful necromancer threatens to raise an army of undead, and you must gather allies to stop their dark ambitions.",
        "output": "Unite a group of brave allies, including warriors, mages, and clerics, to confront the necromancer and prevent the cataclysmic rise of the undead."
    },
    {
        "input": "You awaken with no memory in a mysterious dungeon and must uncover your past while battling deadly creatures.",
        "output": "Explore the depths of the dungeon, regain your memories, and confront the creatures that lurk in the shadows, all while unraveling the mystery of your forgotten past."
    },
    {
        "input": "A group of mischievous goblins has stolen an important artifact, and you are tasked with retrieving it before chaos ensues.",
        "output": "Track down the mischievous goblins, outwit their traps, and recover the stolen artifact to prevent the impending chaos in the realm."
    },
    {
        "input": "You find a hidden map leading to an ancient treasure buried in a treacherous mountain range.",
        "output": "Embark on a perilous expedition through the treacherous mountain range, braving harsh climates and dangerous creatures in search of the long-lost ancient treasure."
    },
    {
        "input": "A cursed forest is spreading its dark influence, and you must find a way to lift the curse and restore the land's vitality.",
        "output": "Venture into the cursed forest, battling twisted creatures and uncovering the source of the curse, in order to restore the land's natural balance and lift the dark influence."
    },
    {
        "input": "You awaken as a prisoner aboard a pirate ship, and you must navigate the treacherous waters, outsmart the crew, and escape to freedom.",
        "output": "Outwit the pirate crew, navigate treacherous waters, and gather allies to orchestrate a daring escape from the pirate ship and forge your path to freedom."
    },
    {
        "input": "A village is plagued by a malevolent spirit, and you are called upon to investigate and put an end to its haunting.",
        "output": "Explore the haunted village, uncover its dark history, and confront the malevolent spirit, employing your wit, courage, and knowledge of the supernatural to bring peace back to the community."
    },
    {
        "input": "You are tasked with retrieving a stolen artifact from a notorious thieves' guild, navigating their underground hideouts and outsmarting their skilled members.",
        "output": "Infiltrate the thieves' guild's secret hideouts, overcome their cunning traps, and confront their skilled members to retrieve the stolen artifact and restore justice."
    },
    {
        "input": "A powerful wizard's tower appears in the realm, tempting adventurers with its secrets and untold treasures.",
        "output": "Embark on a perilous journey to the wizard's tower, facing magical challenges, solving intricate puzzles, and discovering the hidden treasures and knowledge guarded within."
    },
    {
        "input": "A long-lost prophecy reveals that you are the key to preventing an ancient evil from awakening and plunging the world into darkness.",
        "output": "Accept your role as the prophesied hero, gather allies, and embark on a quest to locate powerful artifacts and confront the ancient evil before it awakens."
    },
    {
        "input": "You encounter a celestial being who offers to bestow a divine blessing upon you, granting extraordinary powers.",
        "output": "Accept the celestial being's offer and undergo a divine transformation, unlocking extraordinary powers and embarking on a mission to bring hope and light to a world in need."
    },
    {
        "input": "A mysterious artifact grants you the ability to manipulate time, opening up possibilities to alter events and reshape destinies.",
        "output": "Harness the power of time manipulation, carefully navigating its consequences, as you embark on a journey to rewrite history, correct past mistakes, and shape the future."
    },
    {
        "input": "You find a hidden portal that transports you to the feywild, a realm of enchantment and whimsy.",
        "output": "Step through the hidden portal and immerse yourself in the wonders of the feywild, encountering magical creatures, solving riddles, and navigating the unpredictable nature of the realm."
    },
    {
        "input": "A cursed mask with the power to possess its wearer falls into your hands, and you must find a way to break its hold and free its victims.",
        "output": "Embark on a quest to break the curse of the possessed mask, seeking ancient rituals, powerful artifacts, or the aid of divine entities to free its victims and neutralize its malevolent influence."
    },
    {
        "input": "You are chosen as the champion of a legendary deity, granted divine weapons and armor to vanquish evil and restore balance.",
        "output": "Embrace your role as the champion of the legendary deity, wield the divine weapons and armor bestowed upon you, and face formidable adversaries to restore balance and protect the realms from darkness."
    },
    {
        "input": "A group of powerful wizards seeks apprentices, and you must prove yourself worthy through a series of magical trials.",
        "output": "Participate in the magical trials set by the group of wizards, demonstrating your spellcasting abilities, problem-solving skills, and knowledge of arcane arts to earn a place as their apprentice."
    },
    {
        "input": "You stumble upon a hidden sanctuary of druids who possess ancient nature magic and wisdom.",
        "output": "Seek acceptance among the druids in the hidden sanctuary, learning their ancient nature magic, connecting with the natural world, and becoming a guardian of balance and harmony."
    },
    {
        "input": "A cursed kingdom is in desperate need of a true ruler, and you are chosen to reclaim the throne and restore prosperity.",
        "output": "Embark on a quest to reclaim the throne of the cursed kingdom, rallying allies, reclaiming lost artifacts, and leading the people towards a new era of prosperity."
    },
    {
        "input": "You are transported to a realm trapped in eternal night, where your mission is to restore light and banish the darkness.",
        "output": "Journey through the realm trapped in eternal night, seeking ways to restore light, unraveling the origin of the darkness, and bringing hope to the inhabitants."
    },
    {
        "input": "A celestial comet streaks across the sky, granting magical abilities to those who witness its arrival.",
        "output": "Harness the newfound magical abilities bestowed upon you by the celestial comet, exploring its origins, honing your powers, and unraveling the cosmic mysteries it holds."
    },
    {
        "input": "You encounter a band of noble knights on a quest to retrieve a stolen artifact and restore honor to their order.",
        "output": "Join forces with the noble knights on their quest to recover the stolen artifact, battling villains, upholding chivalry, and restoring honor to their esteemed order."
    },
    {
        "input": "You awaken as a ghostly apparition and must unravel the mystery of your own death to find peace and move on.",
        "output": "Investigate the circumstances of your death as a ghostly apparition, interacting with the living, solving puzzles, and seeking closure to find peace and transcend to the afterlife."
    },
    {
        "input": "A powerful enchantress offers to teach you the secrets of ancient spells and rituals in exchange for a great sacrifice.",
        "output": "Accept the enchantress's offer and delve into the study of ancient spells and rituals, grappling with the moral dilemmas posed by the sacrifices required to unlock their immense power."
    },
    {
        "input": "You find yourself trapped in a labyrinthine dungeon filled with deadly traps, and your only hope is to find the way out.",
        "output": "Navigate the labyrinthine dungeon, evading deadly traps, solving puzzles, and discovering hidden passages, all in a race against time to find the elusive exit."
    },
    {
        "input": "A legendary phoenix, said to possess the power of rebirth, appears before you, offering you a chance at redemption and renewal.",
        "output": "Embrace the opportunity presented by the legendary phoenix, embarking on a transformative journey of self-discovery, redemption, and embracing the power of renewal."
    },
    {
        "input": "You are transported to a realm where music is magic, and you must master the art of melody to overcome challenges.",
        "output": "Immerse yourself in the realm where music holds magical power, honing your musical abilities, composing harmonies, and using the power of melody to overcome obstacles and shape the world."
    },
    {
        "input": "A legendary sword, said to be capable of slaying gods, is discovered, and you must decide its fate.",
        "output": "Face the moral dilemma of what to do with the legendary sword capable of slaying gods, weighing the consequences of its use and determining the destiny of such a powerful artifact."
    },
    {
        "input": "You stumble upon an enchanted forest inhabited by sentient animals, each with their own stories and quests.",
        "output": "Explore the enchanted forest and engage with the sentient animals, listening to their tales, undertaking quests, and forging meaningful connections with these magical creatures."
    },
    {
        "input": "A powerful curse transforms you into a mythical creature, and you must find a way to reverse the transformation.",
        "output": "Embark on a quest to reverse the powerful curse that has transformed you into a mythical creature, seeking ancient remedies, magical artifacts, or the assistance of mystical beings."
    },
    {
        "input": "You inherit a mysterious heirloom that grants you glimpses into the future, but its visions come at a great cost.",
        "output": "Navigate the complexities of the mysterious heirloom's visions of the future, grappling with the sacrifices and consequences that accompany the knowledge it provides."
    },
    {
        "input": "You discover a hidden society of shape-shifters, and they offer to teach you their ancient art of transformation.",
        "output": "Embrace the teachings of the hidden society of shape-shifters, learning their ancient art of transformation, and unlocking the ability to assume different forms, each with its unique strengths and abilities."
    },
    {
        "input": "A magical phenomenon causes inanimate objects to come to life, and you must restore order before chaos ensues.",
        "output": "Navigate the unpredictable realm where inanimate objects come to life, seeking to restore order and balance, and solving the mysteries behind the magical phenomenon."
    },
    {
        "input": "You encounter a mythical creature on the brink of extinction, and you must protect it from hunters and preserve its existence.",
        "output": "Become the guardian of the endangered mythical creature, fending off hunters, raising awareness, and ensuring its survival and restoration within the natural world."
    },
    {
        "input": "You awaken in a parallel dimension where technology and magic coexist, and you must learn to navigate this unique realm.",
        "output": "Adapt to the parallel dimension where technology and magic intertwine, embracing the fusion of both worlds, and using your knowledge to uncover its secrets and harness its potential."
    },
    {
        "input": "A group of ancient spirits seeks a vessel to carry out their unfinished business, and you are chosen as their conduit.",
        "output": "Accept the role as the conduit for the ancient spirits, assisting them in completing their unfinished business, navigating realms both mortal and ethereal to bring closure and resolution to their stories."
    }
]

import json

with open("dataset.json", "w", encoding="utf8") as fp:
    json.dump(dataset, fp, indent=4)

In [4]:
pprint(dataset[200])

{'input': 'You stumble upon an ancient prophecy foretelling the rise of a '
          'chosen hero to defeat an ancient evil.',
 'output': 'Embrace your role as the chosen hero and gather allies to confront '
           'and vanquish the ancient evil threatening the realm.'}


In [5]:
MODEL_NAME = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable parameters: {trainable_params} || total parameters: {all_params} || trainable%: {100 * trainable_params / all_params}")

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
prompt = f"""
<dm>: You wake up in a mysterious forest.
<roc>: 
""".strip()

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
data = load_dataset("json", data_files="dataset.json")

In [ ]:
def generate_output(data_point):
    return f"""
<dm>: {data_point["input"]}
<roc>: {data_point["output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_output(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

In [ ]:
OUTPUT_DIR = 'experiments'

In [ ]:
!pip install tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir eperiments/runs

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=20,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=100,
    output_dir=OUTPUT_DIR,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard"
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
model.push_to_hub("andrea-coppari/qlora-roc", use_auth_token=True)

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [6]:
PEFT_MODEL = "andrea-coppari/qlora-roc"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=False,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
def generate_response(input: str) -> str:
    prompt = f"""
<dm>: {input}
<roc>: 
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    roc_start = "<roc>:"
    response_start = response.find(roc_start)
    response =  response[response_start + len(roc_start) :].strip()
    response_end = response.find("<")
    return response[:response_end].strip()

In [9]:
%%time
device = "cuda:0"

prompt = "You get teleported into a magical kingdom surrounded by giant red mushshrooms and green tubes, everything seems unreal and a curious little creature is approaching you"
print(generate_response(prompt))

Greet the little creature and see if it offers any assistance or guidance, the magical kingdom is filled with unusual creatures and their knowledge may prove invaluable
CPU times: user 50.2 s, sys: 490 ms, total: 50.7 s
Wall time: 50.7 s


In [10]:
%%time
device = "cuda:0"

prompt = "A curse has turned all of the town’s children into chickens, but no one knows which chickens are children and which are just chickens."
print(generate_response(prompt))

Investigate the town and identify the children among the chickens, unraveling the curse and restoring normalcy.
CPU times: user 48.6 s, sys: 99.9 ms, total: 48.7 s
Wall time: 48.7 s


In [11]:
%%time
device = "cuda:0"

prompt = "Within a sunken ruin, a glowing amulet hums with untapped power. Retrieve it or face the wrath of ancient guardians."
print(generate_response(prompt))

Claim the glowing amulet from the ancient ruins, unlocking its untapped power, facing the wrath of the ancient guardians, and embarking on a quest to wield the amulet's newfound power responsibly.
CPU times: user 48.9 s, sys: 73.6 ms, total: 48.9 s
Wall time: 49 s


In [12]:
%%time
device = "cuda:0"

prompt = "In a mystical forest, a mischievous creature leads you towards the lost city of Eldara. Follow or forge your own path? Choose wisely, adventurer."
print(generate_response(prompt))

Follow the creature into the unknown and uncover the secrets of the lost city.
CPU times: user 50.1 s, sys: 43.3 ms, total: 50.1 s
Wall time: 50.1 s


In [13]:
%%time
device = "cuda:0"

prompt = "In a treacherous mountain pass, a legendary dragon guards a hidden treasure. Will you dare to challenge its might or seek another path to riches?"
print(generate_response(prompt))

Face the mighty dragon in a battle of wits and strength, seeking to claim its treasure or forge an unlikely alliance.
CPU times: user 50 s, sys: 35.5 ms, total: 50 s
Wall time: 50 s


In [15]:
%%time
device = "cuda:0"

prompt = "Deep in a cursed swamp, a forgotten temple holds a powerful artifact. Beware the treacherous spirits that haunt its halls. Will you claim the relic or succumb to the swamp's malevolent grasp?"
print(generate_response(prompt))

Venture into the cursed swamp, facing the ancient spirits that guard the powerful artifact, and unravel the artifact's dark history and purpose.
CPU times: user 50.3 s, sys: 50.9 ms, total: 50.4 s
Wall time: 50.4 s


In [16]:
%%time
device = "cuda:0"

prompt = "A mysterious portal opens, revealing a realm of floating islands. To uncover the secret of their levitation, brave the perils and solve the riddles that guard the ancient knowledge."
print(generate_response(prompt))

Venture into the floating islands, solving puzzles, uncovering hidden knowledge, and unraveling the secrets of their levitation.
CPU times: user 50.6 s, sys: 34.3 ms, total: 50.6 s
Wall time: 50.6 s


In [17]:
%%time
device = "cuda:0"

prompt = "On the outskirts of a forgotten village, an ominous mist conceals a hidden graveyard. Uncover the truth behind the restless spirits or risk being trapped in their eternal torment."
print(generate_response(prompt))

Explore the haunted graveyard and unravel the mysteries of the restless spirits. Navigate through the mist, facing the challenges of the afterlife, and uncover the secrets of the forgotten village.
CPU times: user 50.7 s, sys: 23.1 ms, total: 50.8 s
Wall time: 50.8 s
